# Image Offsets: Create offset arrays used by drizzle

The offset tables created by a previous notebook (Offsets_2) are used to generate the X and Y offset arrays used by drizzle.

Each offset table will generate 2 arrays, for X and Y respectively, stored as FITS image of short float type.

The algorithms were developed in the Timing notebook; here, they are cast as callable functions used in a loop to process all images in the sequence.

In [1]:
import os, glob
import numpy as np

import multiprocessing as mp
from multiprocessing import Pool

from astropy.table import Table

import rawpy

In [2]:
datadir = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/'

## Functions

In [3]:
# Comparison functions
gt_zero = lambda x: x > 0.0
lt_zero = lambda x: x < 0.0

# Gets the index of the closest star in table. 
# The differences are in the sense pixel - star centroid.
# The comparison functions define from which quadrant the star is drawn from.
def closest(diff_x, diff_y, compare_x, compare_y):
    # Compute mask that masks out everything that is outside 
    # the quadrant defined by the comparison functions
    mask_x = np.where(compare_x(diff_x), 1, 0)
    mask_y = np.where(compare_y(diff_y), 1, 0)
    mask = mask_x * mask_y

    # Get index of star at minimum distance
    distance = np.sqrt((diff_x * diff_x + diff_y * diff_y)) * mask
    if np.nonzero(distance)[0].size > 0:
        mindist = np.min(distance[np.nonzero(distance)])
        index = np.where(distance == mindist )[0][0]
        return index, mindist
    else:
        return -1, 0.0

## Read last table in sequence, and prototype image

Starting with the last table ensures that we get always the same stars along the entire sequence. Offset tables at the beginning of the sequence may include stars that are dropped later on.

In [4]:
# last table in sequence
table_list = glob.glob(datadir + '/*.offsets_table.fits')
table_list.sort()
last = table_list[-1]
offsets_table = Table.read(last)

In [5]:
# prototype image (used to define array size)
image_name = last.split('/')[-1]
image_name = image_name.replace('.offsets_table.fits', '.ARW')
image_name = os.path.join(datadir, image_name)
raw = rawpy.imread(image_name)
imarray = raw.raw_image_visible.astype(float)

ny = imarray.shape[0]
nx = imarray.shape[1]

print(nx, ny)

4256 2848


## Parallelization functions

The offset computation for each individual pixel is prohibitive without parallelization. 

In [6]:
class Worker:
    '''
    A class with callable instances that execute the offset calculation
    algorithm over a section of the input image. It provides the callable 
    for the `Pool.apply_async` function, and also holds all parameters 
    necessary to perform the calculation.
    '''
    def __init__(self, x0, y0, size_x, size_y, step_x, step_y, centroid_x, centroid_y, 
                offset_x, offset_y, offset_array_x, offset_array_y):
        '''
        Parameters:
        
        x0, y0 - top left pixel of the image section designated for this instance
        size_x, size_ - size of the image section
        step_x - step used in the x direction when looping over pixels
        step_y - step used in the y direction when looping over pixels
        centroid_x - 1-D data from the `xcentroid` column in the offsets table 
        centroid_y - 1-D data from the `ycentroid` column in the offsets table 
        offset_x - 1-D data from the `xoffset` column in the offsets table 
        offset_y - 1-D data from the `yoffset` column in the offsets table 
        
        Returns:
        
        offset_array_x - output array with the x offsets for every `step_x` pixel
        offset_array_y - output array with the y offsets for every `step_y` pixel
        '''
        self.x0 = x0
        self.y0 = y0
        self.size_x = size_x
        self.size_y = size_y
        self.step_x = step_x
        self.step_y = step_y

        self.centroid_x = centroid_x
        self.centroid_y = centroid_y
        self.offset_x = offset_x
        self.offset_y = offset_y
        self.offset_array_x = offset_array_x
        self.offset_array_y = offset_array_y

    def __call__(self):
        for i in range(self.x0, self.size_x, self.step_x):
            for j in range(self.y0, self.size_y, self.step_y):
        # for i in range(2000, 2001):
        #     for j in range(1000, 1003):

                pixel_x = i
                pixel_y = j

                diff_x = pixel_x - self.centroid_x
                diff_y = pixel_y - self.centroid_y

                index = np.array(range(4), dtype=int)
                dist  = np.array(range(4), dtype=float)

                # get index and distance of the closest star, one per quadrant
                index[0], dist[0] = closest(diff_x, diff_y, gt_zero, gt_zero)
                index[1], dist[1] = closest(diff_x, diff_y, lt_zero, gt_zero)
                index[2], dist[2] = closest(diff_x, diff_y, gt_zero, lt_zero)
                index[3], dist[3] = closest(diff_x, diff_y, lt_zero, lt_zero)

                # need a cleanup here. Negative indices, zeroed distances.

                # weighted average of the offset values. The weight is the inverse distance pixel-star.
                sumweights = 1./dist[0] + 1./dist[1] + 1./dist[2] + 1./dist[3]

                weighted_offset_x = self.offset_x[index[0]] * (1./dist[0] / sumweights) + \
                                    self.offset_x[index[1]] * (1./dist[1] / sumweights) + \
                                    self.offset_x[index[2]] * (1./dist[2] / sumweights) + \
                                    self.offset_x[index[3]] * (1./dist[3] / sumweights)
                weighted_offset_y = self.offset_y[index[0]] * (1./dist[0] / sumweights) + \
                                    self.offset_y[index[1]] * (1./dist[1] / sumweights) + \
                                    self.offset_y[index[2]] * (1./dist[2] / sumweights) + \
                                    self.offset_y[index[3]] * (1./dist[3] / sumweights)

                self.offset_array_x[j][i] = weighted_offset_x
                self.offset_array_y[j][i] = weighted_offset_y

## Create output arrays and get data from offsets table

In [7]:
offset_array_x = np.asarray(imarray) * 0.0
offset_array_y = np.asarray(imarray) * 0.0

In [8]:
centroid_x = offsets_table['xcentroid'].data
centroid_y = offsets_table['ycentroid'].data
offset_x = offsets_table['xoffset'].data
offset_y = offsets_table['yoffset'].data

## Populate output arays

In [9]:
# w = Worker(0, 0, nx, ny, 10, 10, centroid_x, centroid_y, offset_x, offset_y, offset_array_x, offset_array_y)


w1 = Worker(0, 0, int(nx/2), ny, 5, 5, centroid_x, centroid_y, offset_x, offset_y, offset_array_x, offset_array_y)
w2 = Worker(int(nx/2)+1, ny, int(nx/2), ny, 5, 5, centroid_x, centroid_y, offset_x, offset_y, offset_array_x, offset_array_y)

w1()
w2()

/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:73: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: divide by zero encountered in double_sc

KeyboardInterrupt: 

In [ ]:
offset_array_x[1940:1960,1940:1960]